In [1]:
import os
import re
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
sns.set('paper','whitegrid', font_scale=2, rc={"lines.linewidth": 2, 'grid.linestyle': '--'})

In [2]:
def get_summary_perfomance(local_dir: str, expt_name: str = None):
    summary_performance = {}
    expt_names = glob.glob(os.path.join(local_dir, "*_*"))
    for expt_name in expt_names:
        algo, env_params, timelog = expt_name.split("\\")[-1].split("__")
        trials = glob.glob(os.path.join(expt_name, "ExperimentCV_*"))
        summary_trials = {}
        for trial in trials:
            with open(os.path.join(trial, "params.json"), 'r') as f:
                params = json.load(f)
            train_start = datetime.strptime(params["_train_start"], "%Y-%m-%d") + relativedelta(years=params["__trial_index__"])
            eval_start = datetime.strptime(params["_train_start"], "%Y-%m-%d") + relativedelta(years=params["__trial_index__"] + params["_train_years"])
            column_name = f"{train_start.year}(+{params['_train_years']}) | {eval_start.year}(+{params['_eval_years']})"
            
            try:
                perfomance_train = pd.read_csv(os.path.join(trial, "best-stats-train", "performance.csv"), index_col=0).rename(columns={'0': column_name})
                perfomance_eval = pd.read_csv(os.path.join(trial, "best-stats-eval", "performance.csv"), index_col=0).rename(columns={'0': column_name})
                summary_trials[column_name] = perfomance_eval
            except Exception as e:
                pass
        
        summary_performance[f"{algo}_{env_params}"] = summary_trials

    return summary_performance

In [3]:
# summary_perfomance = get_summary_perfomance("./ray_results/compare-action-3")
summary_perfomance = get_summary_perfomance("./ray_results/compare-stop_loss-3")

In [4]:
performance_expt = pd.DataFrame()
for expt_name, summary_trials in summary_perfomance.items():
    performance_cv = pd.DataFrame()
    for trial, performance in summary_trials.items():
        performance = performance.iloc[-4:, :].astype(float)
        performance_cv = pd.concat([performance_cv, performance], axis=1)


    performance_cv_mean = performance_cv.mean(axis=1)
    performance_cv_mean.name = expt_name
    performance_expt = pd.concat([performance_expt, performance_cv_mean], axis=1)
    
performance_expt

,A2C_default,A2C_stop_loss=True,DQN_default,DQN_stop_loss=True,PPO_default,PPO_stop_loss=True,SAC_default,SAC_stop_loss=True
Return (Ann.) [%],21.405177,17.729602,20.024521,28.778047,4.629291,7.935853,24.935914,15.546683
Volatility (Ann.) [%],13.291070,12.248378,11.742665,11.866047,12.031112,12.694153,11.365627,13.088591
Sharpe Ratio,1.485703,1.391085,1.555774,2.081625,0.497786,0.659098,1.819155,1.278602
Max. Drawdown [%],-9.472761,-7.503878,-6.043798,-7.264270,-8.492493,-10.254774,-7.434627,-9.905008
